## setup libraries

In [22]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (102 kB/s)
Reading package lis

In [23]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt data for a given date range

Note: this example only fetches the events table. The gdelt python API also can pull from the mentions and gkg tables. See https://linwoodc3.github.io/gdeltPyR/ for full API details



In [24]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

In [25]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()
lst_data = []

# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)


def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='gkg',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
          return filename
    except:
        print("Error occurred")

# test gkg table
#Before the start of the epidemic
test_data1 = [get_filename(x) for x in pd.date_range('2019 Feb 1','2019 May 1')]
#On February 11, 2020, the World Health Organization officially 
#announced the name of the disease as "2019 coronavirus disease".
test_data2 = [get_filename(x) for x in pd.date_range('2020 Feb 1','2020 May 1')]
#The epidemic continues
test_data3 = [get_filename(x) for x in pd.date_range('2021 Feb 1','2021 May 1')]

dbg(test_data3)
#downloading
Before_Epidemics = list(e.map(intofile,test_data1))
Start_Epidemics = list(e.map(intofile,test_data2))
Still_Epidemics = list(e.map(intofile,test_data3))

dbg(Still_Epidemics)

['20210201_gdeltdata.csv', '20210202_gdeltdata.csv', '20210203_gdeltdata.csv', '20210204_gdeltdata.csv', '20210205_gdeltdata.csv', '20210206_gdeltdata.csv', '20210207_gdeltdata.csv', '20210208_gdeltdata.csv', '20210209_gdeltdata.csv', '20210210_gdeltdata.csv', '20210211_gdeltdata.csv', '20210212_gdeltdata.csv', '20210213_gdeltdata.csv', '20210214_gdeltdata.csv', '20210215_gdeltdata.csv', '20210216_gdeltdata.csv', '20210217_gdeltdata.csv', '20210218_gdeltdata.csv', '20210219_gdeltdata.csv', '20210220_gdeltdata.csv', '20210221_gdeltdata.csv', '20210222_gdeltdata.csv', '20210223_gdeltdata.csv', '20210224_gdeltdata.csv', '20210225_gdeltdata.csv', '20210226_gdeltdata.csv', '20210227_gdeltdata.csv', '20210228_gdeltdata.csv', '20210301_gdeltdata.csv', '20210302_gdeltdata.csv', '20210303_gdeltdata.csv', '20210304_gdeltdata.csv', '20210305_gdeltdata.csv', '20210306_gdeltdata.csv', '20210307_gdeltdata.csv', '20210308_gdeltdata.csv', '20210309_gdeltdata.csv', '20210310_gdeltdata.csv', '20210311_g

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


...

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

['20210201_gdeltdata.csv', '20210202_gdeltdata.csv', '20210203_gdeltdata.csv', '20210204_gdeltdata.csv', '20210205_gdeltdata.csv', '20210206_gdeltdata.csv', '20210207_gdeltdata.csv', '20210208_gdeltdata.csv', '20210209_gdeltdata.csv', '20210210_gdeltdata.csv', '20210211_gdeltdata.csv', '20210212_gdeltdata.csv', '20210213_gdeltdata.csv', '20210214_gdeltdata.csv', '20210215_gdeltdata.csv', '20210216_gdeltdata.csv', '20210217_gdeltdata.csv', '20210218_gdeltdata.csv', '20210219_gdeltdata.csv', '20210220_gdeltdata.csv', '20210221_gdeltdata.csv', None, '20210223_gdeltdata.csv', '20210224_gdeltdata.csv', '20210225_gdeltdata.csv', '20210226_gdeltdata.csv', '20210227_gdeltdata.csv', '20210228_gdeltdata.csv', '20210301_gdeltdata.csv', '20210302_gdeltdata.csv', '20210303_gdeltdata.csv', '20210304_gdeltdata.csv', '20210305_gdeltdata.csv', '20210306_gdeltdata.csv', '20210307_gdeltdata.csv', '20210308_gdeltdata.csv', None, '20210310_gdeltdata.csv', '20210311_gdeltdata.csv', '20210312_gdeltdata.csv',

In [26]:
#!zip -o articles.zip *.csv

In [27]:
def find_date(results):
#Find useful dates
  result = []
  for i in results:
    if i != None:
      result.append(i)
  return result

result_Before = find_date(Before_Epidemics)
result_Start = find_date(Start_Epidemics)
result_Still = find_date(Still_Epidemics)
print("Before >>>>>>>>>>    ", result_Before)
print("Start >>>>>>>>>>>    ", result_Start)
print("Still >>>>>>>>>>>    ", result_Still)

Before >>>>>>>>>>     ['20190201_gdeltdata.csv', '20190202_gdeltdata.csv', '20190203_gdeltdata.csv', '20190204_gdeltdata.csv', '20190205_gdeltdata.csv', '20190206_gdeltdata.csv', '20190207_gdeltdata.csv', '20190208_gdeltdata.csv', '20190209_gdeltdata.csv', '20190210_gdeltdata.csv', '20190211_gdeltdata.csv', '20190212_gdeltdata.csv', '20190213_gdeltdata.csv', '20190215_gdeltdata.csv', '20190216_gdeltdata.csv', '20190217_gdeltdata.csv', '20190218_gdeltdata.csv', '20190219_gdeltdata.csv', '20190220_gdeltdata.csv', '20190221_gdeltdata.csv', '20190222_gdeltdata.csv', '20190223_gdeltdata.csv', '20190224_gdeltdata.csv', '20190225_gdeltdata.csv', '20190226_gdeltdata.csv', '20190227_gdeltdata.csv', '20190228_gdeltdata.csv', '20190301_gdeltdata.csv', '20190302_gdeltdata.csv', '20190303_gdeltdata.csv', '20190304_gdeltdata.csv', '20190305_gdeltdata.csv', '20190306_gdeltdata.csv', '20190307_gdeltdata.csv', '20190308_gdeltdata.csv', '20190309_gdeltdata.csv', '20190310_gdeltdata.csv', '20190311_gdelt

## read downloaded files into RDDs

In [28]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

test_data1 = sqlContext.read.option("header", "true").csv(result_Before)
test_data2 = sqlContext.read.option("header", "true").csv(result_Start)
test_data3 = sqlContext.read.option("header", "true").csv(result_Still)

#dbg(test_data)
test_data_rdd1 = test_data1.rdd
test_data_rdd2 = test_data2.rdd
test_data_rdd3 = test_data3.rdd


dbg(test_data_rdd1)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[('20190225234500-0', '20190225234500'), ('20190225234500-1', '20190225234500'), ('20190225234500-2', '20190225234500'), ('20190225234500-3', '20190225234500'), ('20190225234500-4', '20190225234500'), ('20190225234500-5', '20190225234500'), ('20190225234500-6', '20190225234500'), ('20190225234500-7', '20190225234500'), ('20190225234500-8', '20190225234500'), ('20190225234500-9', '20190225234500'), ('20190225234500-10', '20190225234500'), ('20190225234500-11', '20190225234500'), ('20190225234500-12', '20190225234500'), ('20190225234500-13', '20190225234500'), ('20190225234500-14', '20190225234500'), ('20190225234500-15', '20190225234500'), ('20190225234500-16', '20190225234500'), ('20190225234500-17', '20190225234500'), ('20190225234500-18', '20190225234500'), ('20190225234500-19', '20190225234500'), ('20190225234500-20', '20190225234500'), ('20190225234500-21', '20190225234500'), ('20190225234500-22', '20190225234500'), ('20190225234500-23', '20190225234500'), ('20190225234500-24', '20

In [29]:
test_data1.show()

+-----------------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|      GKGRECORDID|          DATE|SourceCollectionIdentifier|    SourceCommonName|  DocumentIdentifier|              Counts|            V2Counts|              Themes|            V2Themes|           Locations|         V2Locations|             Persons|           V2Persons|       Organizations|     V2Organizations|              V2Tone|               Dates|                GCAM|        SharingImage|       RelatedImages|   SocialImageEmbeds|

## countryName_And_Theme

> RDD that countain all the country fields and themes fields without split

# Country_And_Theme

> this RDD contain countryCode with a list Theme with each artical




In [30]:
#take country and theme as rdd 
countryName_And_Theme1 = test_data_rdd1.map(lambda a: (a.Locations, a.V2Themes))
countryName_And_Theme2 = test_data_rdd2.map(lambda a: (a.Locations, a.V2Themes))
countryName_And_Theme3 = test_data_rdd3.map(lambda a: (a.Locations, a.V2Themes))

dbg(countryName_And_Theme1)

[('2#New York, United States#US#USNY#42.1497#-74.9384#NY;1#Canada#CA#CA#60#-96#CA;2#New Jersey, United States#US#USNJ#40.314#-74.5089#NJ;2#Massachusetts, United States#US#USMA#42.2373#-71.5314#MA', 'BAN,1856;INFRASTRUCTURE_BAD_ROADS,3038;WB_1458_HEALTH_PROMOTION_AND_DISEASE_PREVENTION,3038;WB_135_TRANSPORT,3038;WB_1429_ROAD_SAFETY,3038;WB_790_TRANSPORT_AND_SUSTAINABLE_DEVELOPMENT,3038;WB_635_PUBLIC_HEALTH,3038;WB_784_TRAFFIC_AND_ROAD_SAFETY,3038;WB_1808_TRANSPORT_SAFETY,3038;UNGP_TRANSPORTATION_ROADS,3038;TAX_FNCACT_LEADERS,1541;ECON_TAXATION,370;ECON_TAXATION,1041;ECON_TAXATION,1788;ECON_TAXATION,1804;ECON_TAXATION,2564;EPU_POLICY_TAX,370;EPU_POLICY_TAX,1041;EPU_POLICY_TAX,1788;EPU_POLICY_TAX,1804;EPU_POLICY_TAX,2564;EPU_CATS_TAXES,370;EPU_CATS_TAXES,1041;EPU_CATS_TAXES,1788;EPU_CATS_TAXES,1804;EPU_CATS_TAXES,2564;EPU_ECONOMY_HISTORIC,344;EPU_ECONOMY_HISTORIC,1424;EPU_ECONOMY_HISTORIC,2594;CRIME_ILLEGAL_DRUGS,120;CRIME_ILLEGAL_DRUGS,722;CRIME_ILLEGAL_DRUGS,795;CRIME_ILLEGAL_DRUGS,1070

In [31]:
#make country as a key
def find_country(string):
  """spilt the str and find the country"""
  if string == None:
    return str("Not found")
  lst = string.split('#')
  if len(lst) >= 3:
    return lst[2]
  else:
    return str("Not found")
take_country1 = countryName_And_Theme1.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")
take_country2 = countryName_And_Theme2.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")
take_country3 = countryName_And_Theme3.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")

dbg(take_country1)

[('US', 'BAN,1856;INFRASTRUCTURE_BAD_ROADS,3038;WB_1458_HEALTH_PROMOTION_AND_DISEASE_PREVENTION,3038;WB_135_TRANSPORT,3038;WB_1429_ROAD_SAFETY,3038;WB_790_TRANSPORT_AND_SUSTAINABLE_DEVELOPMENT,3038;WB_635_PUBLIC_HEALTH,3038;WB_784_TRAFFIC_AND_ROAD_SAFETY,3038;WB_1808_TRANSPORT_SAFETY,3038;UNGP_TRANSPORTATION_ROADS,3038;TAX_FNCACT_LEADERS,1541;ECON_TAXATION,370;ECON_TAXATION,1041;ECON_TAXATION,1788;ECON_TAXATION,1804;ECON_TAXATION,2564;EPU_POLICY_TAX,370;EPU_POLICY_TAX,1041;EPU_POLICY_TAX,1788;EPU_POLICY_TAX,1804;EPU_POLICY_TAX,2564;EPU_CATS_TAXES,370;EPU_CATS_TAXES,1041;EPU_CATS_TAXES,1788;EPU_CATS_TAXES,1804;EPU_CATS_TAXES,2564;EPU_ECONOMY_HISTORIC,344;EPU_ECONOMY_HISTORIC,1424;EPU_ECONOMY_HISTORIC,2594;CRIME_ILLEGAL_DRUGS,120;CRIME_ILLEGAL_DRUGS,722;CRIME_ILLEGAL_DRUGS,795;CRIME_ILLEGAL_DRUGS,1070;CRIME_ILLEGAL_DRUGS,1866;CRIME_ILLEGAL_DRUGS,1992;CRIME_ILLEGAL_DRUGS,2690;CRIME_ILLEGAL_DRUGS,3196;WB_1921_PRIVATE_SECTOR_DEVELOPMENT,1717;WB_405_BUSINESS_CLIMATE,1717;WB_2531_INSPECTIONS_

In [32]:
#make theme as a list
def to_list(string):
  if string == None:
    return str("No Theme")
  else:
    res = string.split(";")
    result = []
    for i in res[:-1]:
      word = i.split(",")
      result.append(word[0])
    return set(result)
Country_And_Theme1 = take_country1.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")
Country_And_Theme2 = take_country2.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")
Country_And_Theme3 = take_country3.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")

dbg(Country_And_Theme1)

[('US', {'LEGALIZE', 'ECON_UNIONS', 'TAX_FNCACT_MAJORITY_LEADER', 'WB_1921_PRIVATE_SECTOR_DEVELOPMENT', 'WB_470_EDUCATION', 'TAX_FNCACT_POLICE', 'WB_405_BUSINESS_CLIMATE', 'EDUCATION', 'WB_2433_CONFLICT_AND_VIOLENCE', 'WB_2165_HEALTH_EMERGENCIES', 'WB_1331_HEALTH_TECHNOLOGIES', 'WB_2167_PANDEMICS', 'WB_696_PUBLIC_SECTOR_MANAGEMENT', 'EPU_POLICY_LAW', 'WB_1458_HEALTH_PROMOTION_AND_DISEASE_PREVENTION', 'EPU_POLICY_TAX', 'TAX_FNCACT_PRESIDENT', 'EPU_ECONOMY_HISTORIC', 'JUSTICE', 'TAX_FNCACT_LEADERS', 'USPEC_POLITICS_GENERAL1', 'WB_2530_BUSINESS_ENVIRONMENT', 'LEGISLATION', 'WB_784_TRAFFIC_AND_ROAD_SAFETY', 'WB_2082_LAW_ENFORCEMENT', 'NATURAL_DISASTER_FLOODED', 'CRISISLEX_CRISISLEXREC', 'TAX_DISEASE_EPIDEMIC', 'WB_621_HEALTH_NUTRITION_AND_POPULATION', 'CRIME_ILLEGAL_DRUGS', 'TAX_FNCACT_SPEAKER', 'MEDIA_MSM', 'ALLIANCE', 'WB_135_TRANSPORT', 'EPU_POLICY_BUDGET', 'WB_1014_CRIMINAL_JUSTICE', 'WB_635_PUBLIC_HEALTH', 'WB_328_FINANCIAL_INTEGRITY', 'WB_1920_FINANCIAL_SECTOR_DEVELOPMENT', 'WB_1350_

In [33]:
type(Country_And_Theme1)

pyspark.rdd.PipelinedRDD

# Start to anlysis

#   ** function for finding Frequent_Itemsets**



## def Frequent_Itemsets(Country_And_Theme, countryCode)：

> input: countryCode for country to Analysis and a data set

> ouptut: individual, apr_stage2, n

> ***Return:***
*   individual : item with Number of occurrences in basket
*   apr_stage2 ： item pairs Number of occurrences in basket
*   n ： The whole size of a basket







In [34]:

#put it into function

def Frequent_Itemsets(Country_And_Theme, countryCode):
  """remember to change the frequent_num"""
  frequent_num = 50 #define at least how many occurrences are considered frequent

  Themes_of_Country = Country_And_Theme.filter(lambda a: a[0] == countryCode).map(lambda a: a[1])
  n = Themes_of_Country.count()
  #find frequence set and broadcast
  individual = Themes_of_Country.flatMap(lambda a:[(i,1) for i in a]).reduceByKey(lambda a,b:a+b)
  individual1 = individual.filter(lambda a: a[1] >= frequent_num).collectAsMap()   #change the frequence here !!!!!!!!
  individual_broad = sc.broadcast(individual1)
  def select(line):
    res = []
    for i,word1 in enumerate(line):
      if word1 in individual_broad.value:
        for j, word2 in enumerate(line):
          if i > j and word2 in individual_broad.value:
            res.append(tuple(sorted([word1, word2])))
    return res
  #find frequence pairs
  apr_stage2 = Themes_of_Country.flatMap(lambda a:select(a)).map(lambda a:(a, 1)).reduceByKey(lambda a,b:a+b)
  return individual, apr_stage2, n, individual_broad

#individual_Pairs_CH, Frequent_Themes_in_CH, count_of_total_pairs, individual_broad= Frequent_Itemsets("CH")
#dbg(individual_Pairs_CH)
#dbg(Frequent_Themes_in_CH)
#dbg(count_of_total_pairs)

In [35]:
individual_Pairs_CH1, Frequent_Themes_in_CH1, count_of_total_pairs_CH1, individual_broad_CH1= Frequent_Itemsets(Country_And_Theme1, "CH")
individual_Pairs_CH2, Frequent_Themes_in_CH2, count_of_total_pairs_CH2, individual_broad_CH2= Frequent_Itemsets(Country_And_Theme2, "CH")
individual_Pairs_CH3, Frequent_Themes_in_CH3, count_of_total_pairs_CH3, individual_broad_CH3= Frequent_Itemsets(Country_And_Theme3, "CH")

In [36]:
dbg(individual_Pairs_CH3) 
dbg(Frequent_Themes_in_CH3)
dbg(count_of_total_pairs_CH3)
dbg(individual_broad_CH3)

[('TAX_DISEASE_TRAUMA', 21), ('UNREST_BELLIGERENT', 233), ('EPU_POLICY_GOVERNMENT', 904), ('UNGP_FORESTS_RIVERS_OCEANS', 1215), ('INEQUALITY', 25), ('WB_1459_IMMUNIZATIONS', 502), ('WB_1502_EDUCATIONAL_DECENTRALIZATION', 66), ('CRISISLEX_O02_RESPONSEAGENCIESATCRISIS', 61), ('EDUCATION', 684), ('EPU_POLICY_SPENDING', 155), ('TAX_FNCACT_CHIEF_OF_STAFF', 29), ('EPU_POLICY_CONGRESSIONAL', 88), ('WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY', 133), ('TAX_FNCACT_VICE_PRESIDENT', 147), ('WB_1907_PERSONAL_PROPERTY_RIGHTS', 14), ('KILL', 603), ('WB_532_BIOFUELS_ENERGY', 5), ('TAX_FNCACT_CHIEF_FINANCIAL_OFFICER', 35), ('TAX_FNCACT_PHOTOGRAPHER', 6), ('TAX_FNCACT_DRIVER', 52), ('TAX_FNCACT_PEERS', 30), ('WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR', 151), ('TAX_FNCACT_SUPPORTERS', 52), ('TAX_WORLDLANGUAGES_TIBETAN', 9), ('TAX_FNCACT_REPRESENTATIVE', 82), ('TAX_FNCACT_EPIDEMIOLOGIST', 13), ('WB_410_BUSINESS_LAW_AND_REGULATION', 13), ('TAX_FNCACT_EXECUTIVES', 47), ('EPU_POLICY_BUDGET', 66), ('TAX_FNCAC

# 2. computing the support， confidence and Interest rate


## def support(Frequent_pairs , n):

> Frequent_pairs: item Pairs with Number of occurrences in basket

> total count of Baskets items of the country
  
> ***Return***: support rate 


## def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):

> individual_pairs: item with Number of occurrences in basket

> Frequent_Themes： item pairs Number of occurrences in basket

> total count of Baskets items of the country

> ***Return***: confidence

## def interest_rate(individual, confidence):

> individual : item with Number of occurrences in basket

> confidence : Pairs with confidence rate

> ***Return***: interest_rate






In [37]:
def support(Frequent_pairs , n):
  """/"""
  Support_For_Pairs = Frequent_pairs.map(lambda a:(a[0], a[1]/n))
  return Support_For_Pairs
support_CH1 = support(Frequent_Themes_in_CH1, count_of_total_pairs_CH1)
support_CH2 = support(Frequent_Themes_in_CH2, count_of_total_pairs_CH2)
support_CH3 = support(Frequent_Themes_in_CH3, count_of_total_pairs_CH3)

In [38]:
def indi_support(individual_pairs, count_of_total_pairs):
  """/"""
  sup_individual = individual_pairs.map(lambda a:(a[0], a[1]/count_of_total_pairs))
  return sup_individual

In [39]:
def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):
  """..."""
  sup_individual = individual_pairs.map(lambda a:(a[0], a[1]/count_of_total_pairs))
  sup_ij = Frequent_Themes.map(lambda a:(a[0][0], (a[0][1], a[1]/count_of_total_pairs)))
  sup_ji = Frequent_Themes.map(lambda a:(a[0][1], (a[0][0], a[1]/count_of_total_pairs)))
  sup_all_ij_and_ji = sup_ij.union(sup_ji)
  #dbg(sup_all_ij_and_ji)
  all = sup_all_ij_and_ji.join(sup_individual)
  #dbg(all)
  confidence = all.map(lambda a: ((a[0], a[1][0][0]), a[1][0][1]/a[1][1]))
  return confidence

confidence_CH1 = confidence(individual_Pairs_CH1, Frequent_Themes_in_CH1, count_of_total_pairs_CH1)
confidence_CH2 = confidence(individual_Pairs_CH2, Frequent_Themes_in_CH2, count_of_total_pairs_CH2)
confidence_CH3 = confidence(individual_Pairs_CH3, Frequent_Themes_in_CH3, count_of_total_pairs_CH3)


#dbg(confidence_CH1)
#dbg(confidence_CH1.takeOrdered(5,lambda a:-a[1]))

In [40]:
def interest_rate(individual, confidence, n):
  individual_sup = individual.map(lambda a:(a[0], a[1]/n))
  interest_join = confidence.map(lambda a:(a[0][1], a)).join(individual_sup)
  interest = interest_join.map(lambda a: (a[1][0][0], a[1][0][1] - a[1][1]))
  return interest

interest_CH1  = interest_rate(individual_Pairs_CH1, confidence_CH1, count_of_total_pairs_CH1)
interest_CH2  = interest_rate(individual_Pairs_CH2, confidence_CH2, count_of_total_pairs_CH2)
interest_CH3  = interest_rate(individual_Pairs_CH3, confidence_CH3, count_of_total_pairs_CH3)

In [41]:
# dbg(support_CH1.takeOrdered(5, lambda a:-a[1]))
# dbg(confidence_CH1.takeOrdered(5,lambda a:-a[1]))
# dbg(interest_CH1.takeOrdered(5,lambda a:a[1]))
# dbg(interest_CH1.takeOrdered(5,lambda a:-a[1]))

In [42]:
# dbg(support_CH2.takeOrdered(5, lambda a:-a[1]))
# dbg(confidence_CH2.takeOrdered(5,lambda a:-a[1]))
# dbg(interest_CH2.takeOrdered(5,lambda a:a[1]))
# dbg(interest_CH2.takeOrdered(5,lambda a:-a[1]))

In [43]:
# dbg(support_CH3.takeOrdered(5, lambda a:-a[1]))
# dbg(confidence_CH3.takeOrdered(5,lambda a:-a[1]))
# dbg(interest_CH3.takeOrdered(5,lambda a:a[1]))
# dbg(interest_CH3.takeOrdered(5,lambda a:-a[1]))

# ***Result***

In [44]:
print(count_of_total_pairs_CH1)
print(count_of_total_pairs_CH2)
print(count_of_total_pairs_CH3)

4025
7008
2984


In [45]:
dbg(individual_Pairs_CH1.takeOrdered(10, lambda a: -a[1]))
dbg(individual_Pairs_CH2.takeOrdered(10, lambda a: -a[1]))
dbg(individual_Pairs_CH3.takeOrdered(10, lambda a: -a[1]))

[('EPU_ECONOMY_HISTORIC', 1902), ('TAX_ETHNICITY_CHINESE', 1634), ('TAX_WORLDLANGUAGES_CHINESE', 1634), ('USPEC_POLITICS_GENERAL1', 1554), ('LEADER', 1494), ('UNGP_FORESTS_RIVERS_OCEANS', 1451), ('USPEC_POLICY1', 1449), ('GENERAL_GOVERNMENT', 1447), ('CRISISLEX_CRISISLEXREC', 1395), ('WB_696_PUBLIC_SECTOR_MANAGEMENT', 1340)]
[('WB_621_HEALTH_NUTRITION_AND_POPULATION', 5018), ('TAX_DISEASE_CORONAVIRUS', 4831), ('GENERAL_HEALTH', 4789), ('WB_635_PUBLIC_HEALTH', 4383), ('WB_2165_HEALTH_EMERGENCIES', 4100), ('MEDICAL', 4093), ('WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE', 4037), ('WB_2167_PANDEMICS', 4037), ('CRISISLEX_CRISISLEXREC', 3859), ('UNGP_HEALTHCARE', 3454)]
[('WB_621_HEALTH_NUTRITION_AND_POPULATION', 1394), ('GENERAL_HEALTH', 1382), ('USPEC_POLITICS_GENERAL1', 1260), ('CRISISLEX_CRISISLEXREC', 1243), ('TAX_ETHNICITY_CHINESE', 1224), ('TAX_WORLDLANGUAGES_CHINESE', 1224), ('LEADER', 1222), ('UNGP_FORESTS_RIVERS_OCEANS', 1215), ('UNGP_HEALTHCARE', 1183), ('EPU_ECONO

In [46]:
dbg(indi_support(individual_Pairs_CH1, count_of_total_pairs_CH1).takeOrdered(5, lambda a: -a[1]))
dbg(indi_support(individual_Pairs_CH2, count_of_total_pairs_CH2).takeOrdered(5, lambda a: -a[1]))
dbg(indi_support(individual_Pairs_CH3, count_of_total_pairs_CH3).takeOrdered(5, lambda a: -a[1]))

[('EPU_ECONOMY_HISTORIC', 0.4725465838509317), ('TAX_ETHNICITY_CHINESE', 0.40596273291925467), ('TAX_WORLDLANGUAGES_CHINESE', 0.40596273291925467), ('USPEC_POLITICS_GENERAL1', 0.38608695652173913), ('LEADER', 0.37118012422360247)]
[('WB_621_HEALTH_NUTRITION_AND_POPULATION', 0.7160388127853882), ('TAX_DISEASE_CORONAVIRUS', 0.6893550228310502), ('GENERAL_HEALTH', 0.6833618721461188), ('WB_635_PUBLIC_HEALTH', 0.6254280821917808), ('WB_2165_HEALTH_EMERGENCIES', 0.5850456621004566)]
[('WB_621_HEALTH_NUTRITION_AND_POPULATION', 0.4671581769436997), ('GENERAL_HEALTH', 0.4631367292225201), ('USPEC_POLITICS_GENERAL1', 0.4222520107238606), ('CRISISLEX_CRISISLEXREC', 0.4165549597855228), ('TAX_ETHNICITY_CHINESE', 0.4101876675603217)]


In [47]:
dbg(support_CH1.takeOrdered(5, lambda a:-a[1]))
dbg(support_CH2.takeOrdered(5, lambda a:-a[1]))
dbg(support_CH3.takeOrdered(5, lambda a:-a[1]))

[(('TAX_ETHNICITY_CHINESE', 'TAX_WORLDLANGUAGES_CHINESE'), 0.4057142857142857), (('LEADER', 'USPEC_POLITICS_GENERAL1'), 0.311055900621118), (('EPU_POLICY_GOVERNMENT', 'GENERAL_GOVERNMENT'), 0.3003726708074534), (('LEADER', 'TAX_FNCACT_PRESIDENT'), 0.2732919254658385), (('EPU_ECONOMY_HISTORIC', 'USPEC_POLICY1'), 0.26881987577639754)]
[(('WB_621_HEALTH_NUTRITION_AND_POPULATION', 'WB_635_PUBLIC_HEALTH'), 0.6254280821917808), (('GENERAL_HEALTH', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.6242865296803652), (('TAX_DISEASE_CORONAVIRUS', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.6090182648401826), (('WB_2165_HEALTH_EMERGENCIES', 'WB_635_PUBLIC_HEALTH'), 0.5850456621004566), (('WB_2165_HEALTH_EMERGENCIES', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.5850456621004566)]
[(('GENERAL_HEALTH', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.4195710455764075), (('TAX_ETHNICITY_CHINESE', 'TAX_WORLDLANGUAGES_CHINESE'), 0.4101876675603217), (('UNGP_HEALTHCARE', 'WB_621_HEALTH_NUTRITION_AND_POPULATIO

In [48]:
dbg(confidence_CH1.takeOrdered(5, lambda a: -a[1]))
dbg(confidence_CH2.takeOrdered(5, lambda a: -a[1]))
dbg(confidence_CH3.takeOrdered(5, lambda a: -a[1]))

[(('ECON_FREETRADE', 'WB_698_TRADE'), 1.0), (('TAX_FNCACT_CHIEF_FINANCIAL_OFFICER', 'TAX_FNCACT_OFFICER'), 1.0), (('TAX_FNCACT_CHIEF_FINANCIAL_OFFICER', 'TAX_MILITARY_TITLE_OFFICER'), 1.0), (('TAX_FNCACT_CHIEF_FINANCIAL_OFFICER', 'TAX_FNCACT_CHIEF'), 1.0), (('EPU_CATS_TAXES', 'USPEC_POLICY1'), 1.0)]
[(('EPU_POLICY_CENTRAL_BANK', 'USPEC_POLICY1'), 1.0), (('EPU_POLICY_CENTRAL_BANK', 'WB_1920_FINANCIAL_SECTOR_DEVELOPMENT'), 1.0), (('EPU_POLICY_CENTRAL_BANK', 'EPU_CATS_MONETARY_POLICY'), 1.0), (('WB_2934_COPPER', 'WB_507_ENERGY_AND_EXTRACTIVES'), 1.0), (('WB_2934_COPPER', 'WB_895_MINING_SYSTEMS'), 1.0)]
[(('WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 1.0), (('WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY', 'WB_1406_DISEASES'), 1.0), (('TAX_FNCACT_VICE_PRESIDENT', 'USPEC_POLITICS_GENERAL1'), 1.0), (('TAX_FNCACT_VICE_PRESIDENT', 'TAX_FNCACT_PRESIDENT'), 1.0), (('TAX_FNCACT_VICE_PRESIDENT', 'LEADER'), 1.0)]


In [49]:
dbg(interest_CH1.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH2.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH3.takeOrdered(5,lambda a:-a[1]))

[(('WB_707_POVERTY_MEASUREMENT_AND_ANALYSIS', 'WB_1152_ECONOMIC_AND_SOCIAL_MOBILITY'), 0.9873291925465838), (('WB_1152_ECONOMIC_AND_SOCIAL_MOBILITY', 'WB_707_POVERTY_MEASUREMENT_AND_ANALYSIS'), 0.9873291925465838), (('TAX_FNCACT_CAPTAIN', 'TAX_MILITARY_TITLE_CAPTAIN'), 0.986832298136646), (('TAX_MILITARY_TITLE_CAPTAIN', 'TAX_FNCACT_CAPTAIN'), 0.986832298136646), (('TAX_RELIGION_MUSLIM', 'TAX_ETHNICITY_MUSLIM'), 0.986583850931677)]
[(('TAX_MILITARY_TITLE_SOLDIERS', 'TAX_FNCACT_SOLDIERS'), 0.992579908675799), (('TAX_MILITARY_TITLE_COMMANDER', 'TAX_FNCACT_COMMANDER'), 0.9924372146118722), (('TAX_WORLDLANGUAGES_GERMAN', 'TAX_ETHNICITY_GERMAN'), 0.9921518264840182), (('TAX_ETHNICITY_GERMAN', 'TAX_WORLDLANGUAGES_GERMAN'), 0.9921518264840182), (('WB_2120_SATELLITES', 'WB_2329_ACCESS_AND_CONNECTIVITY'), 0.9920091324200914)]
[(('TAX_WORLDLANGUAGES_ENGLISH', 'TAX_ETHNICITY_ENGLISH'), 0.9829088471849866), (('TAX_ETHNICITY_ENGLISH', 'TAX_WORLDLANGUAGES_ENGLISH'), 0.9829088471849866), (('WB_2601_TR

In [50]:
dbg(interest_CH1.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH2.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH3.takeOrdered(5,lambda a:a[1]))

[(('TAX_RELIGION_CHURCH', 'EPU_ECONOMY_HISTORIC'), -0.32160318762451656), (('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'EPU_ECONOMY_HISTORIC'), -0.32000421096957576), (('TAX_FNCACT_SCIENTIST', 'WB_698_TRADE'), -0.29928035982009), (('TAX_WORLDLANGUAGES_ETHIOPIAN', 'TAX_ETHNICITY_CHINESE'), -0.29637369182336426), (('TAX_WORLDLANGUAGES_ETHIOPIAN', 'TAX_WORLDLANGUAGES_CHINESE'), -0.29637369182336426)]
[(('TAX_FNCACT_PROSECUTORS', 'TAX_DISEASE_CORONAVIRUS'), -0.4864564721064125), (('WB_1039_PROPERTY_LAWS_AND_REGULATIONS', 'TAX_DISEASE_CORONAVIRUS'), -0.44649787997390733), (('WB_1917_INTELLECTUAL_PROPERTY', 'TAX_DISEASE_CORONAVIRUS'), -0.4434533834867879), (('TAX_FNCACT_OPERATOR', 'TAX_DISEASE_CORONAVIRUS'), -0.4393550228310502), (('WB_376_INNOVATION_TECHNOLOGY_AND_ENTREPRENEURSHIP', 'TAX_DISEASE_CORONAVIRUS'), -0.4115772450532724)]
[(('EPU_POLICY_CENTRAL_BANK', 'TAX_ETHNICITY_CHINESE'), -0.30304481041746456), (('EPU_POLICY_CENTRAL_BANK', 'TAX_WORLDLANGUAGES_CHINESE'), -0.30304481041746456)